In [1]:
import warnings
import time
import numpy as np
import torch
import gpytorch
import botorch
import geometric_kernels
import geometric_kernels.torch
from geometric_kernels.spaces import HypercubeGraph, HammingGraph
from geometric_kernels.kernels import MaternGeometricKernel
from geometric_kernels.frontends.gpytorch import GPyTorchGeometricKernel
from botorch.fit import fit_gpytorch_mll

/opt/miniconda3/envs/geometrik/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO (geometric_kernels): Numpy backend is enabled. To enable other backends, don't forget to `import geometric_kernels.*backend name*`.
INFO (geometric_kernels): We may be suppressing some logging of external libraries. To override the logging policy, call `logging.basicConfig`.
/opt/miniconda3/envs/geometrik/lib/python3.11/site-packages/spherical_harmonics/fundamental_set.py:21: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
INFO (geometric_kernels): Torch backend enabled.


In [2]:
def Ackley(x):
    """
    The Ackley Function. See https://www.sfu.ca/~ssurjano/ackley.html for details.
    """

    num_dims = x.shape[1]
    a = 20
    b = 0.2
    c = 2 * np.pi

    sum1 = (x**2).sum(axis=1)
    sum2 = (np.cos(c * x)).sum(axis=1)

    term1 = -a * np.exp(-b * np.sqrt(sum1 / num_dims))
    term2 = -np.exp(sum2 / num_dims)

    return (term1 + term2 + a + np.exp(1)).reshape(-1, 1)


def generate_data(space, obj_fun, num_train, num_test):
    key = np.random.RandomState(1234)
    key, x_train = space.random(key, num_train)
    key, x_test = space.random(key, num_test)

    y_train = obj_fun(x_train)
    y_test = obj_fun(x_test)

    data = [
        torch.tensor(x, dtype=torch.float64) for x in [x_train, y_train, x_test, y_test]
    ]

    return data


def get_kernel_and_likelihood(kernel):
    scaled_kernel = gpytorch.kernels.ScaleKernel(GPyTorchGeometricKernel(kernel))

    noise_prior = gpytorch.priors.torch_priors.GammaPrior(1.1, 0.05)
    noise_prior_mode = (noise_prior.concentration - 1) / noise_prior.rate
    lik_fct = gpytorch.likelihoods.gaussian_likelihood.GaussianLikelihood(
        noise_prior=noise_prior,
        noise_constraint=gpytorch.constraints.GreaterThan(1e-8),
        initial_value=noise_prior_mode,
    )

    return scaled_kernel, lik_fct


def fit_predict(data, scaled_kernel, lik_fct):
    x_train, y_train, x_test, y_test = data

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model = botorch.models.SingleTaskGP(
            x_train, y_train, covar_module=scaled_kernel, likelihood=lik_fct
        )
        mll_fct = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)

    # Fit
    start = time.time()
    fit_gpytorch_mll(mll=mll_fct)
    end = time.time()
    print(f"training: {end - start:.4f} s")

    # Predict
    model.eval()
    start = time.time()
    y_pred = model(x_test)
    end = time.time()
    print(f"prediction: {end - start:.4f} s")

    # RMSE
    rmse = torch.sqrt(((y_pred.mean - y_test) ** 2).mean())
    print(f"RMSE: {rmse.item():.4f}")

    return model

In [3]:
SPACE_DIM = 10
NUM_TRAIN = 3000
NUM_TEST = 3000

In [4]:
# HypercubeGraph
for fast_matern in [False, True]:
    print(f"fast_matern: {fast_matern}")
    print("")

    hypercube_graph = HypercubeGraph(SPACE_DIM)
    kernel = MaternGeometricKernel(hypercube_graph, fast_matern=fast_matern)

    data = generate_data(hypercube_graph, Ackley, NUM_TRAIN, NUM_TEST)

    scaled_kernel, lik_fct = get_kernel_and_likelihood(kernel)

    model = fit_predict(data, scaled_kernel, lik_fct)

    print("")
    print(f"lengthscale: {model.covar_module.base_kernel.lengthscale.item():.4f}")
    print(f"outputscale: {model.covar_module.outputscale.item():.4f}")
    print("")
    print("-" * 100)
    print("")


fast_matern: False

training: 10.7532 s
prediction: 2.2959 s
RMSE: 0.5953

lengthscale: 5.3399
outputscale: 1.3710

----------------------------------------------------------------------------------------------------

fast_matern: True

training: 37.6229 s
prediction: 1.2217 s
RMSE: 0.5972

lengthscale: 3.8122
outputscale: 0.0225

----------------------------------------------------------------------------------------------------



In [5]:
# HammingGraph
for fast_matern in [False, True]:
    print(f"fast_matern: {fast_matern}")
    print("")

    hypercube_graph = HammingGraph(SPACE_DIM, 2)
    kernel = MaternGeometricKernel(hypercube_graph, fast_matern=fast_matern)

    data = generate_data(hypercube_graph, Ackley, NUM_TRAIN, NUM_TEST)

    scaled_kernel, lik_fct = get_kernel_and_likelihood(kernel)

    model = fit_predict(data, scaled_kernel, lik_fct)

    print("")
    print(f"lengthscale: {model.covar_module.base_kernel.lengthscale.item():.4f}")
    print(f"outputscale: {model.covar_module.outputscale.item():.4f}")
    print("")
    print("-" * 100)
    print("")


fast_matern: False

training: 65.4289 s
prediction: 2.7725 s
RMSE: 0.5869

lengthscale: 3.7185
outputscale: 0.0219

----------------------------------------------------------------------------------------------------

fast_matern: True

training: 50.6489 s
prediction: 1.3516 s
RMSE: 0.5869

lengthscale: 3.7185
outputscale: 0.0219

----------------------------------------------------------------------------------------------------

